In [18]:
! pip install selenium
! pip install pandasql

In [19]:
import pandas as pd
from glob import glob ## to search and list downloaded files
from selenium import webdriver ## to get links and ROI pages
from selenium.webdriver.common.by import By

import numpy as np

from pandasql import sqldf ## make possible to run SQL statements over pandas dataframe equally a PostgreSQL or equivalent environment

import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

## Function to scrape data from Internet giving a URL source -> Save a txt file with all book links from URL

In [134]:
def getLinksPages(): # create chrome driver
    
    print('Scrapping from https://books.toscrape.com/ ...')
    
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("headless")
    chrome_options.add_argument('log-level=3')

    path_to_chromedriver = "chromedriver"

    driver = webdriver.Chrome(executable_path=path_to_chromedriver,options=chrome_options)

    # go to website
    links = open("./links.txt","w")
    
    driver.get(f"https://books.toscrape.com/index.html")
    sizeItems  = int(driver.find_elements_by_xpath(f"//body[@id='default']/div/div/div/div/form/strong[1]")[0].text) ### get number of total itens in bookstore
    pagNumber  = int(driver.find_elements_by_xpath(f"//body[@id='default']/div/div/div/div/form/strong[3]")[0].text) ### number of elements per page
    
    for i in range(1,int(sizeItems/pagNumber)+1):
        driver.get(f"https://books.toscrape.com/catalogue/page-{i}.html")

        size_of_list  = driver.find_elements_by_xpath(f"//body[@id='default']/div/div/div/div/section/div[2]/ol/li")
        
        for li in size_of_list[1:len(size_of_list)]:
            article = li.find_elements(By.TAG_NAME, "article")[0]
            h3 = article.find_element_by_xpath('.//preceding::h3[1]')
            a = h3.find_elements(By.TAG_NAME, "a")
            links.write(str(a[0].get_attribute('href'))+'\n')
           

    links.close()
    driver.close()

## Data manipulation and ETL: Functions to map attributes from html files -> Return a list with all extract books information

In [135]:
def mapAttributesFromScraping(linksPath):
    
    print('Data manipulation and ETL in progress...')

    links = open(linksPath,"r")
    
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("headless")
    chrome_options.add_argument('log-level=3')
    
    path_to_chromedriver = "chromedriver"

    driver = webdriver.Chrome(executable_path=path_to_chromedriver,options=chrome_options)
    
    books = []

    # go to website
    for link in links:
        
        driver.get(link)
        
        try: title = driver.find_elements_by_xpath("//div[@id='content_inner']/article/div/div[2]/h1")[0].text       
        except: title = ''
        
        try: price = float(driver.find_elements_by_xpath("//div[@id='content_inner']/article/div/div[2]/p")[0].text.split('£')[1])
        except: price = 0.0
        
        try: stock = int(driver.find_elements_by_xpath("//div[@id='content_inner']/article/div/div[2]/p")[1].text.split('In stock (')[1].split(' available)')[0])
        except: stock = np.nan
        
        try: rating = driver.find_elements_by_xpath("//div[@id='content_inner']/article/div/div[2]/p")[2].get_property('attributes')[0]['textContent'].split('star-rating ')[1]
        except: rating = np.nan
        
        try: category = driver.find_elements_by_class_name("breadcrumb")[0].text.split(' ')[2]
        except: category = ''
        
        try: description = driver.find_elements_by_xpath("//div[@id='content_inner']/article/p")[0].text
        except: description = ''
        
        if  (rating == "One"):   rating = 1
        elif(rating == "Two"):   rating = 2
        elif(rating == "Three"): rating = 3
        elif(rating == "Four"):  rating = 4
        elif(rating == "Five"):  rating = 5

        
        book = [title, category, description, price, rating, stock]

        if book:
            books.append(book)
            
    driver.close()
    return books

## Execute scraping and ETL functions

In [129]:
getLinksPages() ### function to scrapped data on demand (daily or when requested -- CRON Function)

Scrapping from https://books.toscrape.com/ ...


In [137]:
import psycopg2

global dbName
dbName = 'cayenaDB'

def createDatabase(dbName, user = 'postgres', pwd = 'postgres'):
    try:
        #establishing the connection
        conn = psycopg2.connect(
           database="postgres", user=user, password=pwd, host='127.0.0.1', port= '5432'
        )
        conn.autocommit = True

        #Creating a cursor object using the cursor() method
        cursor = conn.cursor()

        #Preparing query to create a database
        sql = 'CREATE database {};'.format(dbName)

        #Creating a database
        cursor.execute(sql)
        print('Database {} was created...'.format(dbName))
    except psycopg2.Error as e:
        print('Database {} already exists!!'.format(dbName))
        
def insertData(inputData, dbName, user = 'postgres', pwd = 'postgres'):
    
    columns = inputData.columns.tolist()
    columns = [each_string.lower() for each_string in columns]

    sqlCreate =  """DROP TABLE IF EXISTS books;
                    CREATE TABLE books (
                     {}       VARCHAR (256),
                     {}       VARCHAR (256),
                     {}       VARCHAR,
                     {}       NUMERIC, 
                     {}       INTEGER, 
                     {}       INTEGER                   
                    );""".format(columns[0], columns[1], columns[2], columns[3], columns[4], columns[5])
    
    books = mapAttributesFromScraping("./links.txt")

    booksDF = pd.DataFrame(data=books, columns=["BookTitle", "Category", "Description", "Price", "Rating", "Availability"])

    try:
        #establishing the connection
        conn = psycopg2.connect(
           database=str.lower(dbName), user=user, password=pwd, host='127.0.0.1', port= '5432'
        )
        conn.autocommit = True

        #Creating a cursor object using the cursor() method
        cursor = conn.cursor()

        #Querying SQL to create table
        cursor.execute(sqlCreate)
        print('Create table...'.format(dbName))
        
        #Querying SQL to insert data from books
        insertedBooks = 0
        for i in booksDF.index:
            sqlInsert = """INSERT INTO books VALUES('{}','{}','{}',{},{},{})""".format(booksDF.loc[i,'BookTitle'].replace("'", "''"), 
                                                                                  booksDF.loc[i,'Category'].replace("'", "''"), 
                                                                                  booksDF.loc[i,'Description'].replace("'", "''"), 
                                                                                  booksDF.loc[i,'Price'], 
                                                                                  booksDF.loc[i,'Rating'],
                                                                                  booksDF.loc[i,'Availability'])
                        
            try:
                cursor = conn.cursor()

                cursor.execute(sqlInsert)
                insertedBooks+=1
                
            except psycopg2.Error as e:
                print(e)
                
        print("{} Books inserted".format(insertedBooks))
              
    except psycopg2.Error as e:
            print(e)

In [138]:
createDatabase(dbName)
insertData(booksDF, dbName)

Database cayenaDB already exists!!
Data manipulation and ETL in progress...
Create table...
950 Books inserted


## What is the average price of books by rating (in stars)?

In [141]:
def avgPrinceByRating(dbname = 'cayenadb', user = 'postgres', pwd = 'postgres'):
    
    conn = psycopg2.connect(
       database=str.lower(dbName), user=user, password=pwd, host='127.0.0.1', port= '5432'
    )
    conn.autocommit = True

    #Creating a cursor object using the cursor() method
    cursor = conn.cursor()

    #Querying SQL to create table
    result = cursor.execute("""SELECT Rating, ROUND(AVG(price),2) as "AVG Price (£)" 
                          FROM books
                          GROUP BY Rating;""".format(dbname))
                            
    for i in result:
        print(i)
    
    
    return result

## Call function given the dataframe 
avgPrinceByRating(booksDF)

TypeError: 'NoneType' object is not iterable

## How many books have 2 or less copies on a specific day?

In [27]:
def amountCopiesBooks(booksTable):
    result = sqldf("""SELECT  DATE() AS "Currency Date", COUNT(*) as "Books have 2 or less copies"
                        FROM booksTable 
                        WHERE Availability <=2 
                        """)
    
    return result

## Call function given the dataframe 
amountCopiesBooks(booksDF)

,Currency Date,Books have 2 or less copies
0,2022-05-18,106


In [ ]:
## How many books copies 